In [9]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter

In [12]:
consumer_key='JcKYdqtvb4jQvHuvZueqBGlOU'
consumer_secret='1NcufylSpc2sb83oQWv9qhRGWvYqTFs5ylSoKdPkiJqu07l5ch'
access_token='792341533638946817-shNZlMR3hjQiho72cZwdCb120bwleIR'
access_token_secret='DbKTIhxkymH43wiGV5WmabvPEOWwt9spraQlQnHKUg8j8'

In [13]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
auth_api = API(auth)

In [14]:
item = auth_api.get_user('midasIIITD')

In [15]:
print("name: " + item.name)
print("screen_name: " + item.screen_name)
print("description: " + item.description)
print("statuses_count: " + str(item.statuses_count))
print("friends_count: " + str(item.friends_count))
print("followers_count: " + str(item.followers_count))

name: MIDAS IIITD
screen_name: midasIIITD
description: MIDAS is a group of researchers at IIIT-Delhi who study, analyze, and build different multimedia systems for society leveraging multimodal information.
statuses_count: 292
friends_count: 38
followers_count: 163


In [16]:
no_of_tweets = item.statuses_count

In [17]:
tweets = auth_api.user_timeline(screen_name = 'midasIIITD')

In [31]:
status_list = []
for status in tqdm(Cursor(auth_api.user_timeline, screen_name='midasIIITD', tweet_mode="extended").items()):
    status_list.append(status)
print("No of statuses ", str(len(status_list)))

292it [00:16, 17.60it/s]


No of statuses  292


In [40]:
status._json

{'created_at': 'Mon Jul 23 12:53:15 +0000 2018',
 'id': 1021377705084739584,
 'id_str': '1021377705084739584',
 'full_text': 'MIDAS is a group of researchers at IIIT-Delhi who study, analyze, and build different multimedia systems for society leveraging multimodal information. MIDAS stands for Multimodal Digital Media Analysis Lab and it is founded by Dr. Rajiv Ratn Shah. \nURL: https://t.co/sPvbKgXKmg',
 'truncated': False,
 'display_text_range': [0, 278],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/sPvbKgXKmg',
    'expanded_url': 'http://midas.iiitd.edu.in/',
    'display_url': 'midas.iiitd.edu.in',
    'indices': [255, 278]}]},
 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1021355762575073281,
  'id_str': '1021

In [35]:
status_json_lst =[]
for status in status_list:
    status_json_lst.append(status._json)

In [45]:
import json

with open('midas_tweets.jsonl', 'w') as outfile:
    for entry in status_json_lst:
        json.dump(entry, outfile)
        outfile.write('\n')

In [47]:
tweets = []
for line in open('midas_tweets.jsonl', 'r'):
    tweets.append(json.loads(line))

In [50]:
tweet_dict_lst = []
for tweet in tweets:
    dict_tmp = {'Full Text': tweet['full_text'], 'Date and Time':tweet['created_at'],
                'Number of favorites/likes': tweet['favorite_count'], 
                'Number of retweets': tweet['retweet_count'] }
    tweet_dict_lst.append(dict_tmp)

In [51]:
tweet_df = pd.DataFrame(tweet_dict_lst)
tweet_df

,Date and Time,Full Text,Number of favorites/likes,Number of retweets
0,Sat Mar 16 14:06:56 +0000 2019,We have emailed the task details to all shortl...,6,0
1,Sat Mar 16 09:20:29 +0000 2019,IEEE BigMM 2019 - Call for Workshop Proposals....,1,1
2,Sat Mar 16 09:14:58 +0000 2019,"Congratulations! Arijit, Ramit, @debanjanbhucs...",5,2
3,Sat Mar 16 05:13:14 +0000 2019,We will be releasing a very interesting task t...,6,2
4,Wed Mar 13 17:09:44 +0000 2019,RT @hcdiiitd: Last day to register for #Portfo...,0,2
5,Wed Mar 13 04:11:24 +0000 2019,@ACMMM19 @sigmm @TheOfficialACM @acmmmsys @ACM...,1,0
6,Wed Mar 13 04:06:04 +0000 2019,RT @ACMMM19: The paper deadline is approaching...,0,13
7,Tue Mar 12 17:43:44 +0000 2019,RT @kaggle: Bookmark this amazing library of i...,0,69
8,Tue Mar 12 14:37:55 +0000 2019,Awesome members of our @midasIIITD team who ar...,16,4
9,Mon Mar 11 14:58:30 +0000 2019,"@saanidhi @ACMMM19 Before Friday, 15th March.",0,0
